In [103]:
import pandas as pd
import datetime
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

from icalendar import Calendar, Event

In [104]:
sJours = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche']
dJours = {'lundi':datetime.datetime.strptime("02/09/24","%d/%m/%y"),
          'mardi':datetime.datetime.strptime("03/09/24","%d/%m/%y"),
          'mercredi':datetime.datetime.strptime("04/09/24","%d/%m/%y"),
          'jeudi':datetime.datetime.strptime("05/09/24","%d/%m/%y"), 
          'vendredi':datetime.datetime.strptime("06/09/24","%d/%m/%y"), 
          'samedi':datetime.datetime.strptime("07/09/24","%d/%m/%y"),
          'dimanche':datetime.datetime.strptime("08/09/24","%d/%m/%y")}

In [105]:
fEDT="24-EdT-SDL.xlsx"
xEDT=pd.ExcelFile(fEDT)

In [106]:
", ".join(xEDT.sheet_names)

'Département, Licence 1 Sciences du langage, Licence 2 Sciences du langage, Licence 3 Sciences du langage, Master 1 Sciences du langage, Master 2 Langues et langage, Master 2 Plurilinguisme, Master 2 Linguistique A.O, M1 Plurilinguisme DD, M2 Plurilinguisme DD, Master 1 DiFLES ingénieries de , Master 2 DiFLES ingénieries de , Master 1 DD FLE Naples, Master 2 DD Naples, DU EDILE, DU FLES, DU FLES (FAD), DU LSF'

In [107]:
def readEDT(sheetName):
    dfXL=pd.read_excel(xEDT,sheetName,skiprows=2)
    dfXL=dfXL[dfXL.JOUR.notnull()].copy()
    dfXL.rename({" H Début":"deb","Hfin":"fin","N°semaine":"semaines",
                 "Besoin en salle":"équip","Observations":"obs"},axis=1,inplace=True)
    dfXL.columns=[c.lower() for c in dfXL.columns]
    # print(dfXL1.columns)
    dfXL["code"]=dfXL["code"].ffill()
    dfXL["libellé"]=dfXL["libellé"].ffill()
    dfXL["jour"]=pd.Categorical(dfXL["jour"],categories=sJours,ordered=True)
    dfXL.loc[~(dfXL.deb.str.startswith("1")),"deb"]="0"+dfXL.deb
    dfXL.loc[~(dfXL.fin.str.startswith("1")),"fin"]="0"+dfXL.fin
    dfXL.loc[:,"deb"]=dfXL.deb.str.replace("h",":")
    dfXL.loc[:,"fin"]=dfXL.fin.str.replace("h",":")
    dfXL.loc[:,"deb"]=dfXL.deb.str.replace("H",":")
    dfXL.loc[:,"fin"]=dfXL.fin.str.replace("H",":")
    dfXL.loc[dfXL.type=="CM","groupe"]="CM"
    return dfXL["code libellé enseignant groupe jour deb fin semaines équip obs".split(" ")]

In [109]:
dfL1=readEDT("Licence 1 Sciences du langage")
dfL2=readEDT("Licence 2 Sciences du langage")
dfL3=readEDT("Licence 3 Sciences du langage")
dfM1=readEDT("Master 1 Sciences du langage")
dfM2=readEDT("Master 2 Langues et langage")
df=pd.concat([dfL1,dfL2,dfL3,dfM1,dfM2])

/var/folders/dy/h2nycthd7qjbd6qkb5h8t0nm0000gn/T/ipykernel_49789/1292407992.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'CM' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [111]:
df[(df.code.str[0].isin(list("246")))&df.enseignant.str.contains("Labrune")].sort_values(["jour","deb","fin"]).set_index(["jour","deb","fin"])

code  \
jour     deb   fin               
mardi    13:30 15:30  2MNTM413   
               15:30  2MNTM413   
         15:30 18:30    2MNTY1   
mercredi 13:30 15:30   4LNSE11   
         15:30 17:30   4LNSE11   

                                                                libellé  \
jour     deb   fin                                                        
mardi    13:30 15:30  ECUE Données et modèles en linguistique japonaise   
               15:30  ECUE Données et modèles en linguistique japonaise   
         15:30 18:30  UE Théories et modèles en syntaxe et en phonol...   
mercredi 13:30 15:30                                  ECUE Phonologie 1   
         15:30 17:30                                  ECUE Phonologie 1   

                                             enseignant groupe semaines  \
jour     deb   fin                                                        
mardi    13:30 15:30                   Laurence Labrune    NaN        9   
               15:30                   Laurence Labrune    NaN        9   
         15:30 18:30  Nicolas Guilliot/Laurence Labrune     CM       12   
mercredi 13:30 15:30                   Laurence Labrune    NaN       12   
         15:30 17:30                   Laurence Labrune    NaN       12   

                                 équip            obs  
jour     deb   fin                                     
mardi    13:30 15:30                VP   à partir s.4  
               15:30                VP   à partir s.4  
         15:30 18:30                VP  à partir s.4   
mercredi 13:30 15:30  Jean Loiseau 310            NaN  
         15:30 17:30  Jean Loiseau 310            NaN